# More Finite-Horizon Results
#### Unified Model with Multiple Strategic Agents

James Yu, 20 December 2022

In [1]:
from collections import namedtuple #,defaultdict
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import inv as npinv
from numpy.linalg import matrix_power as nppow

In [2]:
def finite_solution(A, delta, c, T, x_0, M = 1):
    n = len(x_0)
    eps = np.finfo(np.float64).eps
    I = np.identity(n)
    eigvals, U = np.linalg.eig(A)
    chi_0 = U.T @ x_0
    D = np.diag(eigvals)
    K_t = I
    K_sequence = [K_t]
    
    for i in range(T): # generate solution matrices
        K_t_new = I + c * delta * (delta * K_t + c * I) @ nppow(npinv(M * delta * K_t + c * I), 2) @ K_t @ nppow(D, 2)
        K_sequence.insert(0, K_t_new)
        #if np.allclose(K_t, K_t_new, rtol = eps, atol = eps) and i >= T: break
        K_t = K_t_new

    chi_ts = [chi_0]
    chi_var = chi_0
    gamma_ts = []
    L_ts = []
    #K_ss = K_sequence[0]
    #L_ss = -inv(B.T @ K_ss @ B + c * I / delta) @ B.T @ K_ss @ A
    
    payoff = 0
    stage_payoffs = []
    discounted_stage_payoffs = []
    cumulative_payoffs = []

    for i, K in enumerate(K_sequence[1:]): # while True
        L_t = -delta * npinv(c * I + M * delta * K) @ K @ D
        L_ts.append(L_t)
        gamma_t = L_t @ chi_var
        gamma_ts.append(gamma_t)
        p = -(chi_var.T @ chi_var + c * gamma_t.T @ gamma_t).item()
        payoff += delta**i * p
        stage_payoffs.append(p)
        discounted_stage_payoffs.append(delta**i * p)
        cumulative_payoffs.append(payoff)
        chi_var_new = D @ chi_var + M * gamma_t
        chi_ts.append(chi_var_new)
        #if np.allclose(chi_var, chi_var_new, rtol = eps, atol = eps): break
        chi_var = chi_var_new
        # i += 1
    
    # NOTE: missing final time T payoff term; payoffs cannot be interpreted unless added in the future
        
    Results = namedtuple("Results", ["chi_ts", "gamma_ts", "K_ts", "L_ts"])
    return Results(chi_ts, gamma_ts, K_sequence, L_ts)#, K_ss, L_ss

Original example:

In [3]:
A = np.array([
    [0.9, 0.1],
    [0.1, 0.9]
])
delta = 0.9
c = 0.5
T = 5-1
x_0 = np.array([[1.0, 1.0]], ndmin = 2).T
M = 3
result = finite_solution(A, delta, c, T, x_0, M = M)
# K_t results
print("Index 1 Index 2")
for K in result.K_ts:
    print(K[0, 0], K[1, 1])

Index 1 Index 2
1.0613566391677058 1.0393063358102146
1.0613566403747612 1.0393063360172352
1.0613561910770124 1.0393062166883937
1.0615234375 1.039375
1.0 1.0


In [4]:
# MK_t results
print("Index 1 Index 2")
for K in result.K_ts:
    print(3 * K[0, 0], 3 * K[1, 1])

Index 1 Index 2
3.1840699175031175 3.1179190074306438
3.1840699211242836 3.1179190080517056
3.184068573231037 3.117918650065181
3.1845703125 3.118125
3.0 3.0


In [5]:
# L_t results
print("Index 1 Index 2")
for L in result.L_ts:
    print(L[0, 0], L[1, 1])

Index 1 Index 2
-0.2838136190357405 -0.22633750538273212
-0.2838136011870781 -0.22633750145258447
-0.28382024427746677 -0.22633976673408818
-0.28125 -0.225


In [6]:
# opinions
eigvals, U = np.linalg.eig(A)
for chi_t in result.chi_ts:
    print(U @ chi_t)
    print()

[[1.]
 [1.]]

[[0.14855914]
 [0.14855914]]

[[0.02206983]
 [0.02206983]]

[[0.00327824]
 [0.00327824]]

[[0.00051222]
 [0.00051222]]



Since the recursive equation forms do not depend on $T$, we will not see any changes if we reduce $T$.

In [7]:
A_symmetric = np.array([ # stubborn
    [0.9, 0.07, 0.03],
    [0.07, 0.58, 0.35],
    [0.03, 0.35, 0.62]
])
result_stubborn = finite_solution(A_symmetric, delta, c, T, np.array([[1.0, 1.0, 1.0]], ndmin = 2).T, M = M)
# K_t results
print("Index 1 Index 2")
for K in result_stubborn.K_ts:
    print([K[0, 0], K[1, 1], K[2, 2]])

Index 1 Index 2
[1.0613566391677058, 1.0445711865173006, 1.0037835007745688]
[1.0613566403747612, 1.044571186858177, 1.003783500774587]
[1.0613561910770126, 1.0445710132679178, 1.0037835006667668]
[1.0615234375, 1.044659415092354, 1.003784139595146]
[1.0, 1.0, 1.0]


In [8]:
result_stubborn.K_ts[0] <= result_stubborn.K_ts[1] # t = 0 vs t = 1, etc

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [9]:
result_stubborn.K_ts[2] <= result_stubborn.K_ts[1] 

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [10]:
result_stubborn.K_ts[2] <= result_stubborn.K_ts[3] 

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [11]:
result_stubborn.K_ts[4] <= result_stubborn.K_ts[3] 

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [13]:
A_balanced = np.array([ # balanced symmetric
    [0.48, 0.3, 0.22],
    [0.3, 0.52, 0.18],
    [0.22, 0.18, 0.6]
])
result_balanced = finite_solution(A_balanced, delta, c, T, np.array([[1.0, 1.0, 1.0]], ndmin = 2).T, M = M)
# K_t results
print("Index 1 Index 2")
for K in result_balanced.K_ts:
    print([K[0, 0], K[1, 1], K[2, 2]])

Index 1 Index 2
[1.0613566391677058, 1.0023193146150768, 1.0101282142244044]
[1.0613566403747612, 1.0023193146150793, 1.0101282142253354]
[1.0613561910770124, 1.0023193145902316, 1.0101282121611452]
[1.0615234375, 1.002319554704756, 1.0101327890452438]
[1.0, 1.0, 1.0]


All of these cases demonstrate periodicity.